# **Installations**

In [1]:
!pip install deepface easyocr gradio regex
!pip install google-generativeai
!pip install opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.7 MB/s eta 0:00:00



# **Imports and Configuration**

In [2]:
import gradio as gr
import easyocr
import numpy as np
import regex as re
import json
import cv2
from datetime import datetime
from deepface import DeepFace
import warnings
import google.generativeai as genai

# Filter out warnings for a cleaner output
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

25-10-26 08:35:07 - Directory /root/.deepface has been created
25-10-26 08:35:07 - Directory /root/.deepface/weights has been created


In [3]:
try:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=GEMINI_API_KEY)

except ImportError:
    print("Not in a Colab environment. Please set the GEMINI_API_KEY manually.")

# Instantiate the Gemini model
llm = genai.GenerativeModel('gemini-2.5-flash')

# **Initialize OCR**

In [4]:
# Initialize the EasyOCR reader
reader = easyocr.Reader(['en'])

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

# **Tool Functions**

In [5]:

def tool_assess_image_quality(state):
    """Checks if the ID card image is blurry. A crucial first step."""
    state['analysis_log'].append("EXECUTING: tool_assess_image_quality")
    gray = cv2.cvtColor(state['id_card_image'], cv2.COLOR_BGR2GRAY)
    blur_score = cv2.Laplacian(gray, cv2.CV_64F).var()

    if blur_score < 100: # This threshold is empirical and may need tuning
        state['id_card_quality']['is_blurry'] = True
        state['status'] = "rejected"
        state['rejection_reason'] = f"ID card image is too blurry (score: {blur_score:.2f}). Please upload a clear image."
        return f"Image quality check failed. Blur score {blur_score:.2f} is below the threshold of 100."
    else:
        state['id_card_quality']['is_blurry'] = False
        return f"Image quality is acceptable. Blur score: {blur_score:.2f}."

def tool_extract_text_from_id(state):
    """Uses OCR to extract raw text from the ID card image."""
    state['analysis_log'].append("EXECUTING: tool_extract_text_from_id")
    try:
        results = reader.readtext(state['id_card_image'])
        extracted_text = "\n".join([result[1] for result in results])
        if not extracted_text.strip():
            state['status'] = 'manual_review_required'
            state['rejection_reason'] = 'OCR failed to find any text on the document.'
            return "OCR failed: No text was found on the document."
        state['extracted_text'] = extracted_text
        return "OCR successful. Extracted text is now in the state."
    except Exception as e:
        state['status'] = 'error'
        state['rejection_reason'] = f"A critical error occurred during OCR: {e}"
        return f"OCR process failed with an exception: {e}"

def tool_parse_kyc_details(state):
    """Parses the raw extracted text using regex to find specific details."""
    state['analysis_log'].append("EXECUTING: tool_parse_kyc_details")
    text = state['extracted_text']
    details = {
        "Name": "Not found",
        "Document Number": "Not found",
        "Date Of Birth": "Not found",
        "Issue Date": "Not found",
        "Address": "Not found",
        "Nationality": "NEPALESE"
    }
    name_match = re.search(r'Full Name\s*\n\s*([A-Z\s]+?)\s*(\n|$)', text, re.IGNORECASE)
    if name_match: details["Name"] = name_match.group(1).strip()

    doc_num_match = re.search(r'Certificate No\s*[:.\\s]*\n?([\d-]+)', text, re.IGNORECASE)
    if doc_num_match: details["Document Number"] = doc_num_match.group(1).strip()

    dob_match = re.search(r'Year\s*:\s*(\d{4})\s*Month\s*:\s*([A-Z]{3})\s*Day\s*\.?\s*(\d{1,2})', text, re.IGNORECASE)
    if dob_match:
        year, month, day = dob_match.groups()
        details["Date Of Birth"] = f"{year}-{month}-{day}"

    issue_date_match = re.search(r'(\d{4}[-.]\\d{2}[-.]\\d{2})', text)
    if issue_date_match: details["Issue Date"] = issue_date_match.group(1).strip()

    address_match = re.search(r'Permanent Address[\s\S]*?District\s*:\s*([A-Za-z]+)', text, re.IGNORECASE)
    if address_match: details["Address"] = "District: " + address_match.group(1).strip()

    state['parsed_details'] = details
    return f"Parsing complete. Found details: {list(details.keys())}"

def tool_verify_faces(state):
    """Compares the face from the ID card with the selfie using DeepFace."""
    state['analysis_log'].append("EXECUTING: tool_verify_faces")
    try:
        result = DeepFace.verify(
            img1_path=state['id_card_image'],
            img2_path=state['selfie_image'],
            model_name='VGG-Face',
            enforce_detection=False
        )
        state['face_verification_result'] = {
            'verified': result['verified'],
            'distance': result.get('distance', 999),
            'message': "Faces Match!" if result['verified'] else "Faces Do Not Match!"
        }
        return f"Face verification complete. Match status: {result['verified']}"
    except Exception as e:
        state['status'] = 'error'
        state['rejection_reason'] = f"A critical error occurred during face verification: {e}"
        return f"CRITICAL ERROR during face verification: {e}"

def tool_calculate_fraud_score_and_conclude(state):
    """Calculates a final risk score and makes a final decision. Should be the last step."""
    state['analysis_log'].append("EXECUTING: tool_calculate_fraud_score_and_conclude")
    score = 0
    indicators = []

    missing_fields = [k for k, v in state['parsed_details'].items() if v == "Not found"]
    if missing_fields:
        score += len(missing_fields) * 15
        indicators.append(f"Could not extract: {', '.join(missing_fields)}")

    if not state.get('face_verification_result', {}).get('verified', False):
        score += 50
        indicators.append("Face verification failed or faces did not match.")

    state['fraud_score'] = min(score, 100)
    state['fraud_indicators'] = indicators

    if state['fraud_score'] >= 40 or not state.get('face_verification_result', {}).get('verified', False):
        state['status'] = 'manual_review_required'
    else:
        state['status'] = 'verified'

    return "Final fraud score calculated and decision concluded."

# **Agent Core Logic**

The Agent's Core Logic
This section contains the brain of our agent.
- **AVAILABLE_TOOLS**: A dictionary that tells the agent what tools it has and what they do. This is crucial for the LLM to make informed decisions.
- **create_prompt**: This function dynamically builds the prompt for Gemini, including the agent's mission, the current state of the KYC case, and the list of available tools.
- **kyc_agent_orchestrator**: The main function that replaces the old linear script. It runs a loop where it asks the LLM for the next action, executes the chosen tool, and updates the state, until the KYC case is resolved.

In [6]:
AVAILABLE_TOOLS = {
    "assess_image_quality": {
        "function": tool_assess_image_quality,
        "description": "Checks the ID card image for quality issues like blur. Should be the very first step."
    },
    "extract_text_from_id": {
        "function": tool_extract_text_from_id,
        "description": "Use OCR to extract all raw text from the ID card. Must be done after a successful image quality check."
    },
    "parse_kyc_details": {
        "function": tool_parse_kyc_details,
        "description": "Parses the raw text extracted by OCR to find specific details like Name, Document Number, etc."
    },
    "verify_faces": {
        "function": tool_verify_faces,
        "description": "Compares the face on the ID card with the user's selfie to see if they match."
    },
    "calculate_fraud_score_and_conclude": {
        "function": tool_calculate_fraud_score_and_conclude,
        "description": "Calculates a final risk score and makes a final decision (verified, manual_review). This should be the absolute last step when all other data has been gathered."
    }
}

In [7]:
def create_prompt(state, tools_manifest):
    # Create a simplified version of the state for the prompt, excluding image data
    prompt_state = {k: v for k, v in state.items() if k not in ['id_card_image', 'selfie_image']}

    # Format the available tools for the prompt
    tools_string = "\n".join([
        f'- `{name}`: {info["description"]}' for name, info in tools_manifest.items()
    ])

    prompt = f"""
You are an expert AI KYC Verification Agent. Your goal is to process a KYC application by deciding the next best action to take based on the current state of the case.

**Current Case State:**
```json
{json.dumps(prompt_state, indent=2)}
```

**Available Tools:**
{tools_string}

**Your Task:**
Based on the current state, what is the single next tool you should use to move the verification process forward?
Your response MUST be a JSON object with two keys:
1. `tool_name`: The name of the tool to use (e.g., "extract_text_from_id").
2. `justification`: A brief (1-2 sentence) reason for your choice.

For example: {{"tool_name": "assess_image_quality", "justification": "The process has just started, so the first step is always to check the quality of the uploaded ID card image."}}
"""
    return prompt

In [8]:
def generate_final_report(state):
    risk_level = "Low"
    if 30 <= state.get('fraud_score', 0) < 60:
        risk_level = " Medium"
    elif state.get('fraud_score', 0) >= 60:
        risk_level = " High"

    parsed_details = state.get('parsed_details', {})
    face_result = state.get('face_verification_result', {})
    fraud_indicators = state.get('fraud_indicators', [])

    report = f"""
--- KYC Verification Agent Report ---
Final Status: {state['status'].upper()}
Timestamp: {datetime.now().isoformat()}

--- Agent's Reasoning Log ---
{chr(10).join(state['analysis_log'])}

--- Summary ---
Extracted Document Data:
  - Name: {parsed_details.get('Name', 'N/A')}
  - Document Number: {parsed_details.get('Document Number', 'N/A')}
  - Date Of Birth: {parsed_details.get('Date Of Birth', 'N/A')}
  - Issue Date: {parsed_details.get('Issue Date', 'N/A')}
  - Address: {parsed_details.get('Address', 'N/A')}
  - Nationality: {parsed_details.get('Nationality', 'N/A')}

Face Verification:
  - Message: {face_result.get('message', 'Not Performed')}
  - Verified: {face_result.get('verified', 'N/A')}

Fraud Detection:
  - Fraud Score: {state.get('fraud_score', 'N/A')}/100
  - Risk Level: {risk_level}
  - Indicators: {chr(10) + '    - '.join(fraud_indicators) if fraud_indicators else 'No significant risk indicators found.'}
"""
    return report

In [9]:
def kyc_agent_orchestrator(id_card, selfie):
    if id_card is None or selfie is None:
        return "Please upload both an ID card and a selfie."

    # 1. Initialize the agent's state (memory)
    state = {
        "id_card_image": id_card,
        "selfie_image": selfie,
        "status": "pending_analysis",
        "analysis_log": ["▶️ KYC process initiated."],
        "extracted_text": None,
        "parsed_details": None,
        "id_card_quality": {},
        "face_verification_result": None,
        "fraud_score": None,
        "rejection_reason": None
    }

    # 2. The Agent's Reasoning Loop
    max_steps = 7 # Safety break to prevent infinite loops
    for step in range(max_steps):
        if state['status'] != 'pending_analysis':
            state['analysis_log'].append(f"⏹️ Process concluded with status: {state['status'].upper()}")
            break

        state['analysis_log'].append(f"\n--- Step {step + 1}: Agent is thinking... ---")

        # 3. Ask the LLM for the next action
        prompt = create_prompt(state, AVAILABLE_TOOLS)
        try:
            response = llm.generate_content(prompt)
            # Clean the response to ensure it's valid JSON
            cleaned_response = response.text.strip().replace('`', '').replace('json', '')
            decision = json.loads(cleaned_response)
            tool_name = decision['tool_name']
            justification = decision['justification']
        except (json.JSONDecodeError, AttributeError, ValueError) as e:
            state['analysis_log'].append(f"ERROR: Could not parse LLM response: {response.text}. Error: {e}")
            state['status'] = 'error'
            continue

        state['analysis_log'].append(f"🧠 Agent chose tool: `{tool_name}`. Justification: {justification}")

        # 4. Execute the chosen tool
        if tool_name in AVAILABLE_TOOLS:
            tool_function = AVAILABLE_TOOLS[tool_name]['function']
            tool_result = tool_function(state)
            state['analysis_log'].append(f"🔧 Tool `{tool_name}` Result: {tool_result}")
        else:
            state['analysis_log'].append(f"ERROR: LLM chose an invalid tool: {tool_name}")
            state['status'] = 'error'
    else:
        state['analysis_log'].append("WARNING: Reached maximum agent steps. Forcing manual review.")
        state['status'] = 'manual_review_required'

    # 5. Generate the final report from the final state
    final_report = generate_final_report(state)
    return final_report

# **Gradio Web Interface**

In [ ]:
iface = gr.Interface(
    fn=kyc_agent_orchestrator,
    inputs=[
        gr.Image(type="numpy", label="Upload ID Card"),
        gr.Image(type="numpy", label="Upload Selfie")
    ],
    outputs=[
        gr.Textbox(label="Verification Agent Report", lines=30)
    ],
    title="AI-Powered KYC Verification Agent (v2 - Gemini Powered)",
    description="Upload an ID card and a selfie. An AI agent powered by Gemini will dynamically choose the best steps to verify your identity, assess risk, and generate a detailed report of its actions."
)

iface.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a3a34298c9454ba0b9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


25-10-26 08:37:49 - 🔗 vgg_face_weights.h5 will be downloaded from https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5 to /root/.deepface/weights/vgg_face_weights.h5...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5
To: /root/.deepface/weights/vgg_face_weights.h5
100%|██████████| 580M/580M [00:19<00:00, 29.2MB/s]
